In [47]:
# Do OSy stuff.
import os
# Pause to let the browser catch up.
from time import sleep
# Browser automation.
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
# Because I'm learning Pony
from pony import orm
# Date stuffs.
import datetime
# Download stufs.
import requests
# https://stackoverflow.com/a/16696317
# Download PDF stuffs.
import re
date_regex = re.compile("([\w]+)\. ([\d]+)[\w]+, ([\d]+)")

def download_pdf(url, local_filename=None):
    # All filenames are just 'Meijer_digital_receipt.pdf', grab the 
    # second to last unique part of the URL.
    if local_filename is None:
        local_filename = url.split('/')[-2]+".pdf"
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

In [72]:
os.unlink('receipts.sqlite')

In [74]:
# Database of receipts.
database = 'receipts.sqlite'
filename = os.path.abspath(database)
db = orm.Database(
    provider='sqlite',
    filename=filename,
    create_db=True,
)
class Receipt(db.Entity):
    date = orm.Required(datetime.datetime)
    store = orm.Required(str)
    items = orm.Required(int)
    total = orm.Required(float)
    pdf   = orm.Required(str)
db.generate_mapping(create_tables=True)

In [26]:
# Cache receipt table in a csv.
csv = open("receipts.csv", "w")

In [27]:
# mPerks user & password on separate lines in .secrets file
with open(".secrets", "r") as fid:
    user, password = fid.read().splitlines()

In [15]:
# Create Firefox driver.
driver = webdriver.Firefox()
# Set wait time to 5s to allow network time.
driver.implicitly_wait(5)
# Go to the mPerks account login.
driver.get("https://accounts.meijer.com/manage/Account/mPerks#/form/user")
# Fill user & submit.
elem = driver.find_element_by_name("email")
elem.clear()
elem.send_keys(user)
elem.send_keys(Keys.RETURN)
# Fill password & submit.
elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(password)
elem.send_keys(Keys.RETURN)
# Wait for login to complete
driver.find_element_by_id("editStore")
# Go to the receipts page
driver.get("https://mperks.meijer.com/mperks/ShoppingTools#receipts")
driver.find_element_by_class_name("receipt-list")

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="05ae1b65-c03f-481d-a135-cc57cd8c9916", element="d8cea618-f939-4c9d-bd56-2d2767f7cdf8")>

In [18]:
# Continually click "View More" until all available receipts are shown.
while True:
    try:
        driver.find_element_by_class_name("viewMore").click()
    except ElementNotInteractableException:
        print("Done.")
        break
    except ElementClickInterceptedException:
        sleep(1)
    except:
        raise

In [19]:
receiptItems = driver.find_elements_by_class_name("receiptItem")

In [20]:
for receiptItem in receiptItems:
    break

In [78]:
def processReceipt(receiptItem):
    date = receiptItem.find_element_by_class_name("c1").text
    store = receiptItem.find_element_by_class_name("c2").text
    items = receiptItem.find_element_by_class_name("c3").text
    total = receiptItem.find_element_by_class_name("c4").text
    receipt_pdf = receiptItem.find_element_by_class_name("c5").find_element_by_tag_name("a").get_attribute("href")
    
    
    date_regex_ = date_regex.findall(date)[0]
    date2 = "{} {} {}".format(y[0], y[1], y[2])
    datetime_date = datetime.datetime.strptime(date2, "%b %d %Y")

    Receipt(
        date = datetime_date,
        store = store,
        items = int(items),
        total = float(total.split("$")[1]),
        pdf = receipt_pdf
    )
    csv.write("{},{},{},{},{}\n".format(date,
                                       store,
                                       items,
                                       total,
                                       receipt_pdf))
    db.commit()

In [79]:
processReceipt(receiptItem)